# info
- by: LeeHaEun
- start: 21.11.08 Mon
- end: 21.11.08 Mon
- review: 21.11.08 Mon

# theme: CNN

## 모델의 성능 올리는 방법
=> **CNN** 사용(convolution neural network)
1. convolution
  - goal: 여러가지 필터를 통해 얻어낸 맵 중, 해당 이미지의 특징을 가장 잘 나타내는 것 find -> 이미지의 특징 파악
  - kernal 이라는 필터를 적용
    - **stride**: 필터를 적용하는 간격
  - 가장자리?
    - 없는 것으로 치거나
    - zero padding 사용: 가장자기에 0인 셀이 있다고 가정
2. maxpooling
  - 영역을 요약해서 입력 이미지의 크기를 줄이는 과정 = **downsampling**
  - 특정 grid 안의 값 중 가장 큰 값을 선택
  - 이렇게 하면 사이즈 절반 됨
  => 기준으로 잡는 그리드의 크기에 따라 달라진다


## terms
- **Convolution layer**
  - dense layer랑 유사
  - karas의 `Conv2D` 레이어 타입으로 사용 가능
  - weight, biases, kernal 포함
  - 필터 매트릭스 내부의 값 = 올바른 출력을 내기 위해 조정되는 변수
  - => 입력받은 이미지에 대한 특징 추출
- **CNN**: 컨볼루션 레이어를 적어도 한 개 이상 가지고 있는 네트워크


# code_l04c01_image_classification_with_cnns

## 기본 세팅

In [ ]:
import tensorflow as tf

# Import TensorFlow Datasets
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

# Helper libraries
import math
import numpy as np
import matplotlib.pyplot as plt

import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

### import MNIST data set

In [ ]:
dataset, metadata = tfds.load('fashion_mnist', as_supervised=True, with_info=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

In [ ]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal',      'Shirt',   'Sneaker',  'Bag',   'Ankle boot']

-> label 이름이 dataset안에 없으니까 저장해두기

### 불러온 데이터를 확인해볼까용

In [ ]:
num_train_examples = metadata.splits['train'].num_examples
num_test_examples = metadata.splits['test'].num_examples
print("Number of training examples: {}".format(num_train_examples))
print("Number of test examples:     {}".format(num_test_examples))

트레이닝 세트 60000개, 테스트 세트 10000개

## 데이터 전처리 (lec 3이랑 same)

In [ ]:
def normalize(images, labels):
  images = tf.cast(images, tf.float32) #이미지를 float으로 캐스팅
  images /= 255 #255로 나누기
  return images, labels

# dataset를 normalize해서 덮어씌우기
train_dataset =  train_dataset.map(normalize)
test_dataset  =  test_dataset.map(normalize)

# 처음 dataset 사용할 때에는 disk에 로드된다 -> caching하면 memory로 불러온다 -> faster!
train_dataset =  train_dataset.cache()
test_dataset  =  test_dataset.cache()

- **tf.cast** = 새로운 형태로 cast
- **map**: 함수를 데이터에 적용
  - → map(함수, 데이터)
  - → 데이터.map(함수)
- **cache()**는 그냥 데이터 저장소

### 전처리 된 데이터를 살펴볼까용

In [ ]:
# Take a single image, and remove the color dimension by reshaping
for image, label in test_dataset.take(1): #testset에서 1개만 불러오기->image,label
  break
image = image.numpy().reshape((28,28)) #image를 넘파이 파입으로 저장

# Plot the image - voila a piece of fashion clothing
plt.figure()
plt.imshow(image, cmap=plt.cm.binary)
plt.colorbar()
plt.grid(False)
plt.show()

take(): 해당 배치(배치작업은, 데이터를 실시간으로 처리하는게 아니라, 일괄적으로 모아서 처리하는 작업을 의미한다.)를 몇 번 불러올지 정한다


cmap은 색깔 차트

plt는 위에서 matplotlib으로 선언했음

In [ ]:
plt.figure(figsize=(10,10))
i = 0
for (image, label) in test_dataset.take(25):
    image = image.numpy().reshape((28,28))
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(image, cmap=plt.cm.binary)
    plt.xlabel(class_names[label])
    i += 1
plt.show()

matplotlib.pyplot 모듈의 subplot(rows, cols, index) 함수는 여러 개의 그래프를 하나의 그림에 나타내도록 합니다.


## 모델 만들기(**layer setting에서 차이가 있음!**)

### 레이어 세팅

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation=tf.nn.relu,
                           input_shape=(28, 28, 1)), # 입력 이미지에 적용되는 필터: 원래 사이즈 유지 32개의 컨볼루션 이미지 생성
    tf.keras.layers.MaxPooling2D((2, 2), strides=2), #stride 2인 풀링 -> 32개 출력의 크기 줄이기
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation=tf.nn.relu), # 입력 이미지, 64개 출력
    tf.keras.layers.MaxPooling2D((2, 2), strides=2), # 마찬가지로 64개 출력의 크기 줄이기
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu), #128개 노드로부터 입력받아
    tf.keras.layers.Dense(10, activation=tf.nn.softmax) #output 노드=10개
])

### 모델 컴파일(로스펑션, 옵티마이저, 메트릭스)

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

## 모델 학습시키기(lec3이랑 same)(**정확도는 더 높다! 0.97**)

In [ ]:
BATCH_SIZE = 32
train_dataset = train_dataset.cache().repeat().shuffle(num_train_examples).batch(BATCH_SIZE)
test_dataset = test_dataset.cache().batch(BATCH_SIZE)

In [ ]:
model.fit(train_dataset, epochs=10, steps_per_epoch=math.ceil(num_train_examples/BATCH_SIZE))

As the model trains, the loss and accuracy metrics are displayed. This model reaches an accuracy of about 0.97 (or 97%) on the training data.

=> lecc3보다 정확도가 높다!

## 모델 정확도 판단(lec3이랑 same)

In [ ]:
test_loss, test_accuracy = model.evaluate(test_dataset, steps=math.ceil(num_test_examples/32))
print('Accuracy on test dataset:', test_accuracy)

## 예측하고 돌려봅시다(lec3이랑 same)

In [ ]:
for test_images, test_labels in test_dataset.take(1):
  test_images = test_images.numpy()
  test_labels = test_labels.numpy()
  predictions = model.predict(test_images)

In [ ]:
predictions.shape #(32,10)
predictions[0] #0번에 관한 예측 확률 제시
np.argmax(predictions[0]) #위의 확률중에 제일 큰거 return
test_labels[0] #실제로 4인것을 확인할 수 있다

In [ ]:
def plot_image(i, predictions_array, true_labels, images): #이미지와 주석
  predictions_array, true_label, img = predictions_array[i], true_labels[i], images[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  
  plt.imshow(img[...,0], cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'
  
  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label): ##막대그래프
  predictions_array, true_label = predictions_array[i], true_label[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)
  
  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

In [ ]:
# 예시 하나만 띄워봅시다
i = 0
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions, test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions, test_labels)

In [ ]:
# 예시 하나만 띄워봅시다
i = 12
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions, test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions, test_labels)

In [ ]:
# 예시 여러개 한번에 띄워봅시다
# Plot the first X test images, their predicted label, and the true label
# Color correct predictions in blue, incorrect predictions in red
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions, test_labels, test_images)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions, test_labels)


In [ ]:
# Grab an image from the test dataset
img = test_images[0]

print(img.shape)

In [ ]:
# Add the image to a batch where it's the only member.
img = np.array([img])

print(img.shape)

In [ ]:
predictions_single = model.predict(img)
print(predictions_single)

plot_value_array(0, predictions_single, test_labels)
_ = plt.xticks(range(10), class_names, rotation=45)

np.argmax(predictions_single[0])